In [13]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from scipy.optimize import minimize

In [14]:
random.seed(1852026)

In [15]:
df = pd.read_excel("dataPoints.xlsx")

In [16]:
data = df.to_numpy()
data_train, data_rest = train_test_split(data, test_size=0.30)
data_test, data_val = train_test_split(data_rest, test_size=0.50)
data_train, data_validate, data_test = np.split(df.sample(frac=1), [int(.7*len(df)), int(.85*len(df))])
data_train = data_train.to_numpy()
data_validate = data_validate.to_numpy()
data_test = data_test.to_numpy()

X_train = np.transpose(data_train[:,0:2])
Y_train = np.transpose(data_train[:,2:])
X_validate = np.transpose(data_validate[:,0:2])
Y_validate = np.transpose(data_validate[:,2:])
X_test = np.transpose(data_test[:,0:2])
Y_test = np.transpose(data_test[:,2:])

In [17]:
n = X_train.shape[0] # Size of the input vector X
n_y = Y_train.shape[0] # Size of the input vector Y
p = X_train.shape[1] # Size of the sample
N = 10 # size of the hidden layer, # neurons, this is just for the moment to have some example

In [18]:
def initialize_parameters(n, N, n_y):
    """
    Argument:
    n -- size of the input layer
    N -- size of the hidden layer
    p -- size of the sample
    n_y -- size of the output layer
    
    Returns:
    params -- python dictionary containing your parameters:
                    W1 -- weight matrix of shape (N, n)
                    b1 -- bias vector of shape (N, 1)
                    V -- weight matrix of shape (n_y, N)
    """
        
    W1 = np.random.randn(N,n)
    b1 = np.zeros(shape=(N, 1))
    V = np.random.randn(n_y,N)
    
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "V": V}
    
    return parameters

In [19]:
def second_norm(omega):
    return np.linalg.norm(omega)**2

In [20]:
def hyp_tan(t,sigma):
    return (2*np.exp(2*sigma*t)-1)/(2*np.exp(2*sigma*t)+1)

In [21]:
def predicting(z0,omega,sigma):
    W = omega[:,1:3]
    V = omega[:,0:1]
    b = omega[:,3:4]
    Y_predicted=[predicting_1_ouput(z0[i],b,W,V,sigma) for i in range((z0.T).shape[1])]#(z0.T).shape[1]
    return np.array(Y_predicted)

In [38]:
def predicting_1_ouput(z0,b,W,V,sigma):
    #print("W")
    #print(W)
    #print("X")
   # print(z0)
    z1= np.dot(W,z0)-b.T
    #print("z1")
    #print(np.sum(V*hyp_tan(z1,sigma)))
    return np.sum(V*hyp_tan(z1,sigma))

In [39]:
params = initialize_parameters(n, N, n_y)
W1 = params['W1']
b1 = params['b1']
V = params['V']
omega0=np.append(np.append(V.T,W1, axis=1), b1, axis=1)

In [36]:
omega0

array([[ 0.29208162, -0.61886267, -0.75095845,  0.        ],
       [-0.2645636 , -1.04685831,  1.58369986,  0.        ],
       [-1.06396157, -1.17123919,  0.22625897,  0.        ],
       [ 0.45870124, -2.39574741, -0.02157824,  0.        ],
       [ 0.21850995,  1.32005357,  0.51104727,  0.        ],
       [ 0.36415071,  1.66642432, -1.6839206 ,  0.        ],
       [-0.937911  ,  0.67226585, -0.18750988,  0.        ],
       [ 1.29803057, -0.75507795, -1.20779857,  0.        ],
       [ 0.917579  ,  0.95895849,  1.70017573,  0.        ],
       [ 1.52964646,  1.57413023,  0.95332683,  0.        ]])

In [41]:
X= X_train.T
y_true = Y_train
number_of_samples = X_train.shape[1]
def reg_training_error(input_list):
    omega =input_list# np.array(input_list[:-2])
    rho=0.00001
    sigma=2
    #rho   = input_list[1] 
    #sigma = input_list[2]
    omega=omega.reshape(N,int(len(omega)/N))
    Y_pred=predicting(X,omega,sigma)
    #print(np.sum((Y_pred-y_true)**2))
    #print(number_of_samples)
    #print(np.sum((Y_pred-y_true)**2)/(2*number_of_samples))
    #print(rho*(second_norm(omega)))
    E = (np.sum((Y_pred-y_true)**2)/(2*number_of_samples) + rho*(second_norm(omega)))
    omega=omega.flatten()
    return E#,Y_pred

In [42]:
res = minimize(reg_training_error, omega0)#,method='Nelder-Mead')

In [43]:
res

      fun: 0.041191475483422356
 hess_inv: array([[ 1.49348975e+01,  2.51674030e+00, -3.06865067e+00, ...,
         1.94281406e+01,  1.41003601e+01, -2.38892648e+01],
       [ 2.51674030e+00,  1.20239661e+04,  4.71950742e+03, ...,
        -8.05367048e+03, -3.08128111e+03,  2.62854609e+03],
       [-3.06865067e+00,  4.71950742e+03,  1.93449719e+03, ...,
        -3.15740985e+03, -1.27900158e+03,  1.03003461e+03],
       ...,
       [ 1.94281406e+01, -8.05367048e+03, -3.15740985e+03, ...,
         6.24591857e+03,  2.48713328e+03, -2.29763824e+03],
       [ 1.41003601e+01, -3.08128111e+03, -1.27900158e+03, ...,
         2.48713328e+03,  1.06910350e+03, -9.44156717e+02],
       [-2.38892648e+01,  2.62854609e+03,  1.03003461e+03, ...,
        -2.29763824e+03, -9.44156717e+02,  9.46806145e+02]])
      jac: array([ 1.18929893e-06, -4.38652933e-07, -3.72529030e-07, -1.84681267e-06,
        1.05286017e-06,  6.73811883e-07,  2.24914402e-07, -2.81259418e-07,
        1.09337270e-06, -2.15973705e-06

In [44]:
res['fun']

0.041191475483422356

In [45]:
res['x']

array([ 2.27808323e-01,  4.40953695e+00,  2.07894979e+00, -1.19650358e+00,
        2.20995552e-01,  2.75750126e+00, -9.06568481e-02,  5.35639398e+00,
        2.23036766e-01,  3.48483773e-01, -4.92324970e-02, -6.50095269e-01,
        2.37088046e-01,  1.08818372e+00,  1.74447972e-01,  8.91198585e-01,
        2.35240167e-01, -8.93580208e-01,  2.18155191e-01,  6.76129772e-01,
        2.20839868e-01, -4.15985878e-01,  2.83353089e-01, -4.90768394e-01,
        2.25939037e-01, -1.08367597e+00,  4.87762567e-01, -2.12453417e+00,
        2.27669627e-01, -3.17097708e+00,  5.69328573e-03,  6.45501934e+00,
        2.14641636e-01,  1.83566906e-01, -4.60525627e-01, -2.89941051e-01,
        2.19472780e-01, -3.73077825e+00, -1.77467692e+00,  1.43532516e+00])

In [46]:
def reg_training_error_val_test(omega,X,y_true,number_of_samples):
    #omega =input_list# np.array(input_list[:-2])
    rho=0.00000001
    sigma=2
    #rho   = input_list[1] 
    #sigma = input_list[2]
    omega=omega.reshape(N,int(len(omega)/N))
    Y_pred=predicting(X,omega,sigma)
    #print(np.sum((Y_pred-y_true)**2))
    #print(number_of_samples)
    #print(np.sum((Y_pred-y_true)**2)/(2*number_of_samples))
    #print(rho*(second_norm(omega)))
    E = (np.sum((Y_pred-y_true)**2)/(2*number_of_samples) + rho*(second_norm(omega)))
    #omega=omega.flatten()
    return E#,Y_pred

In [47]:
reg_training_error_val_test(res['x'],X_validate.T,Y_validate,X_validate.shape[1])

0.040556920207149606

In [48]:
reg_training_error_val_test(res['x'],X_test.T,Y_test,X_test.shape[1])

0.07397354895363535

In [40]:
reg_training_error_val_test(omega0.flatten(),X_train.T,Y_train,X_train.shape[1])

67.08610322093497